## Libraries

In [5]:
import os
import json
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
import ollama
from IPython.display import display, Markdown
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [12]:
def find_tours():
    # Setup
    options = uc.ChromeOptions()
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-blink-features=AutomationControlled')
    # options.add_argument('--headless')  # Turn on if needed
    driver = uc.Chrome(options=options)
    
    driver.get("https://flightio.com/")
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    
    wait = WebDriverWait(driver, 15)
    
    # Final result list
    flights = []
    
    try:
        route_containers = wait.until(EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "div.flex.h-14.w-full.items-center.justify-between.rounded-lg.bg-\\[\\#F7F8FB\\].p-3")
        ))
    
        price_elements = driver.find_elements(By.CSS_SELECTOR, "span.text-slate-900.text-base.font-bold")
        price_list = [p.text.strip() for p in price_elements if p.text.strip()]
    
        for i, route in enumerate(route_containers):
            try:
                cities = route.find_elements(By.CSS_SELECTOR, "span.text-sm")
                if len(cities) < 2:
                    continue
    
                to_city = cities[0].text.strip()
                from_city = cities[1].text.strip()
    
                date = route.find_element(By.CSS_SELECTOR, "span.m\\:text-body-xs\\(p\\).d\\:text-xs").text.strip()
    
                price = price_list[i] if i < len(price_list) else "N/A"
    
                flights.append({
                    "from": from_city,
                    "to": to_city,
                    "date": date.replace('\u200c', ' ').strip(),
                    "price": price
                })
    
            except Exception as e:
                print("Error in route:", e)
    
        driver.quit()
    
        # Output result
        for flight in flights:
            print(flight)
    
    except Exception as outer:
        print("Page structure issue:", outer)
        driver.quit()
    return flights

In [13]:
tours = find_tours()

{'from': 'تهران', 'to': 'استانبول', 'date': 'جمعه 5 اردیبهشت', 'price': '4,635,800'}
{'from': 'تهران', 'to': 'دبی', 'date': 'سه شنبه 2 اردیبهشت', 'price': '7,760,000'}
{'from': 'تهران', 'to': 'مشهد', 'date': 'چهارشنبه 3 اردیبهشت', 'price': '2,500,000'}


In [18]:
def available_tours():
    available_tours = "Available tours:\n"
    for index, item in enumerate(flights):
        if index < len(flights)-1:
            available_tours+=f"{index+1}) from {item['from']} to {item['to']} on {item['date']} and the price is: {item['price']}\n"
        else:
            available_tours+=f"{index+1}) from {item['from']} to {item['to']} on {item['date']} and the price is: {item['price']}"
    return available_tours

In [19]:
print(available_tours())

Available tours:
1) from تهران to استانبول on جمعه 5 اردیبهشت and the price is: 4,635,800
2) from تهران to مشهد on چهارشنبه 3 اردیبهشت and the price is: 2,500,000
3) from تهران to دبی on سه شنبه 2 اردیبهشت and the price is: 7,760,000
